# 📊 **Antigranular** Heart Disease Prediction Contest (ft. **Harvard/OpenDP** and **TPDP**)

🎉 Welcome to a new [Antigranular](https://antigranular.com) contest in collaboration with the [TPDP Workshop](https://tpdp.journalprivacyconfidentiality.org/2024/) and [Harvard's OpenDP Community Meeting](https://opendp.org/)!

🩺 This time, we are focusing on [heart condition detection](https://en.wikipedia.org/wiki/Cardiovascular_disease) using our new [TensorFlow Privacy](https://github.com/tensorflow/privacy) and [Opacus (PyTorch)](https://opacus.ai/) models!

🦜 Any questions? Head over to our [Discord](https://discord.com/invite/KJwApgXs4s)!



## 🏃‍♂️ Getting Started

In this section we will download the antigranular package and login




### 📦 Install Antigranular

This command installs the [Antigranular PyPI Package](https://pypi.org/project/antigranular/) on the local enviroment.


In [63]:
# Install the Antigranular package
!pip install antigranular &> /dev/null

### ✍ Login to the Enclave

Head over to [Competitions](https://www.antigranular.com/competitions) to find your `<user_id>`, `<user_secret>` and the competition's name and copy that command here.

![img](https://docs.antigranular.com/shots/comp_cell.png)

In [64]:
import antigranular as ag
session = ag.login(<client_id>,<client_secret>, competition = "Heart Disease Prediction Hackathon")

Dataset "Heart Disease Prediction Hackathon Dataset" loaded to the kernel as heart_disease_prediction_hackathon_dataset
Key Name                       Value Type     
---------------------------------------------
train_y                        PrivateDataFrame
train_x                        PrivateDataFrame
test_x                         DataFrame      

Connected to Antigranular server session id: e722c7d8-6bf4-49d9-99d4-a21a6b422ef7, the session will time out if idle for 25 minutes
Cell magic '%%ag' registered successfully, use `%%ag` in a notebook cell to execute your python code on Antigranular private python server
🚀 Everything's set up and ready to roll!


### 🤖 Using AG

You can now simply use ``%%ag`` to run code on an enclave! You can always head over to our [Docs](https://docs.antigranular.com/) to learn more about AG, but for now, we can define train and test variables as follows.

In [65]:
%%ag
x_train = heart_disease_prediction_hackathon_dataset["train_x"]
y_train = heart_disease_prediction_hackathon_dataset["train_y"]
x_test = heart_disease_prediction_hackathon_dataset["test_x"]

### 🕵️‍♂️ Exploring data

Exploring data in Antigranular involves spending your epsilon budget, be mindful of your usage but remember that the less epsilon you use, the less accurate your results will get!

In [66]:
%%ag
x_train.info()

+----+----------+-------------+---------------+---------+------------+
|    | Column   | numerical   | categorical   | dtype   | bounds     |
|----+----------+-------------+---------------+---------+------------|
|  0 | age      | True        | False         | int64   | (21, 86)   |
|  1 | sex      | True        | False         | int64   | (0, 1)     |
|  2 | bp       | True        | False         | int64   | (80, 215)  |
|  3 | ch       | True        | False         | int64   | (102, 597) |
|  4 | bs       | True        | False         | int64   | (67, 157)  |
|  5 | phr      | True        | False         | int64   | (62, 222)  |
+----+----------+-------------+---------------+---------+------------+



In [67]:
%%ag
y_train.info()

+----+-----------+-------------+---------------+---------+----------+
|    | Column    | numerical   | categorical   | dtype   | bounds   |
|----+-----------+-------------+---------------+---------+----------|
|  0 | condition | True        | False         | int64   | (0, 1)   |
+----+-----------+-------------+---------------+---------+----------+



In [68]:
%%ag
# We can start by exploring the data, carefully using our epsilon
describe = x_train.describe(eps=0.1)
ag_print(describe)

               age          sex  ...           bs          phr
count  7884.000000  7996.000000  ...  8827.000000  8487.000000
mean     52.870034     0.692559  ...   101.046748   136.386258
std      17.204366     0.318314  ...    13.329600    23.495245
min      21.000000     0.000000  ...    67.000000    62.000000
25%      50.039966     0.000758  ...    99.475241    80.854830
50%      57.084218     0.996751  ...    86.512725   142.780217
75%      63.323252     0.998985  ...   127.916433   175.422090
max      79.148852     0.996507  ...   149.999901   207.072679

[8 rows x 6 columns]



In [69]:
%%ag
# We can start by exploring the data, carefully using our epsilon
describe = y_train.describe(eps=0.1)
ag_print(describe)

         condition
count  8093.000000
mean      0.523698
std       0.401498
min       0.000000
25%       0.010079
50%       0.988495
75%       0.972839
max       0.982828



In [70]:
%%ag
# x_test is a public test set, so we can print it without using epsilon
ag_print(x_test)

      age  sex   bp   ch   bs  phr
0      71    1  128  326   95  117
1      61    1  153  270   98  123
2      59    1  113  236  106  181
3      69    0  109  151  109  108
4      55    0  137  235  101  150
...   ...  ...  ...  ...  ...  ...
1995   60    1  128  261  112  143
1996   50    1  143  216   94  100
1997   64    1  120  172   87  142
1998   56    1  158  294   82  144
1999   69    0  117  559  112  157

[2000 rows x 6 columns]



### 🎈 A quick solution

In this section we evaluate an editorial solution in AG using TensorFlow!

In [71]:
%%ag
import tensorflow as tf
from op_pandas import standard_scaler, PrivateDataFrame
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from op_tensorflow import PrivateKerasModel, PrivateDataLoader


# Normal keras model
seqM = Sequential([
    Dense(128, activation='relu', input_shape=(6,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(16, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# Create DP keras model
dp_model = PrivateKerasModel(model=seqM, l2_norm_clip=1, noise_multiplier=0.2)

# Use a standard (non-DP) optimizer directly from keras.
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# PrivateKerasModel uses similar API as standard Keras
dp_model.compile(
	optimizer = optimizer,
	loss = 'binary_crossentropy',
	metrics = ["accuracy"]
)

/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/site-packages/tensorflow_probability/python/__init__.py:58: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  distutils.version.LooseVersion(required_tensorflow_version)):



In [72]:
%%ag
x_train_scaled = standard_scaler(x_train, eps=.9)
x_train_scaled.info()

+----+----------+-------------+---------------+---------+------------------------------------------+
|    | Column   | numerical   | categorical   | dtype   | bounds                                   |
|----+----------+-------------+---------------+---------+------------------------------------------|
|  0 | age      | True        | False         | float64 | (-3.545866308097974, 3.2749010585300433) |
|  1 | sex      | True        | False         | float64 | (-1.4218549858295675,                    |
|    |          |             |               |         | 0.7003866601290992)                      |
|  2 | bp       | True        | False         | float64 | (-2.9457633272833066, 4.694860585906799) |
|  3 | ch       | True        | False         | float64 | (-2.21494939824283, 5.375164119718769)   |
|  4 | bs       | True        | False         | float64 | (-2.2067474065806176,                    |
|    |          |             |               |         | 3.3586316669552514)              

In [73]:
%%ag
data_loader = PrivateDataLoader(feature_df=x_train_scaled , label_df=y_train, batch_size=256)

In [74]:
%%ag
dp_model.fit(x=data_loader, epochs=100, target_delta=1e-1)

Epoch 1/100

32/32 - 10s - loss: 0.6919 - accuracy: 0.5350 - 10s/epoch - 322ms/step

Epoch 2/100

32/32 - 2s - loss: 0.6497 - accuracy: 0.6420 - 2s/epoch - 72ms/step

Epoch 3/100

32/32 - 2s - loss: 0.5757 - accuracy: 0.7094 - 2s/epoch - 72ms/step

Epoch 4/100

32/32 - 2s - loss: 0.5489 - accuracy: 0.7223 - 2s/epoch - 66ms/step

Epoch 5/100

32/32 - 2s - loss: 0.5250 - accuracy: 0.7495 - 2s/epoch - 69ms/step

Epoch 6/100

32/32 - 2s - loss: 0.4987 - accuracy: 0.7536 - 2s/epoch - 66ms/step

Epoch 7/100

32/32 - 2s - loss: 0.4854 - accuracy: 0.7626 - 2s/epoch - 69ms/step

Epoch 8/100

32/32 - 2s - loss: 0.4758 - accuracy: 0.7710 - 2s/epoch - 69ms/step

Epoch 9/100

32/32 - 2s - loss: 0.4763 - accuracy: 0.7731 - 2s/epoch - 68ms/step

Epoch 10/100

32/32 - 2s - loss: 0.4482 - accuracy: 0.7830 - 2s/epoch - 62ms/step

Epoch 11/100

32/32 - 2s - loss: 0.4306 - accuracy: 0.7962 - 2s/epoch - 66ms/step

Epoch 12/100

32/32 - 2s - loss: 0.4240 - accuracy: 0.8019 - 2s/epoch - 63ms/step

Epoch 13/1

In [75]:
%%ag
x_test_scaler = standard_scaler(PrivateDataFrame(x_test), eps=.9)
y_pred = dp_model.predict(x_test_scaler, label_columns=["output"])

63/63 [==============================] - 1s 10ms/step



In [76]:
%%ag
# Note that the predictions are a float scalar
# so we scale it
def f(x: float) -> float:
  if x > 0.5:
    return 1
  else:
    return 0

y_pred["output"] = y_pred["output"].map(f, output_bounds=(0, 1))

### 📝 Make your submission (Through AG)

Submit a prediction by simply typing `submit_predictions(your_prediction)` to find out how you rank on the leaderboard.

![img](https://www.antigranular.com/static/media/Step%209.8091828f3cff4324fe6d.png)


In [77]:
%%ag
result = submit_predictions(y_pred)

score: {'leaderboard': 0.8884343364580213, 'logs': {'BIN_ACC': 0.9164343364580213, 'LIN_EPS': -0.02800000000000003}}



### 🎉 That's it!

Congrats! You made your first submission to the competition! Now it's time to keep exploring the data and to try to achieve a better score! Here are the next steps:


1.  🏫 Head over to our [Docs](https://docs.antigranular.com/) and discover Opacus (PyTorch), TensorFlow Privacy and other libraries available in Antigranular!
2. 🦜 Any questions? Head over to our [Discord](https://discord.com/invite/KJwApgXs4s)!

We hope you have fun and enjoy the competition!

Best of luck,

Antigranular Team